# 전이학습

In [2]:
# @title gdown
# https://drive.google.com/file/d/1z-HEvwpoTDbef_EowtGp7mUINlDRLFw7/view?usp=sharing
import gdown, zipfile, os

file_id = '1z-HEvwpoTDbef_EowtGp7mUINlDRLFw7'

gdown.download(f'https://drive.google.com/uc?id={file_id}', 'SUV_kor.zip', quiet=False)

dir = 'SUV-classification2'
os.makedirs(dir, exist_ok=True)  # 없으면 생성

with zipfile.ZipFile('SUV_kor.zip', 'r') as z:
    z.extractall(dir)

Downloading...
From (original): https://drive.google.com/uc?id=1z-HEvwpoTDbef_EowtGp7mUINlDRLFw7
From (redirected): https://drive.google.com/uc?id=1z-HEvwpoTDbef_EowtGp7mUINlDRLFw7&confirm=t&uuid=80e1faa9-02be-4766-975e-7c8e5856020f
To: /content/SUV_kor.zip
100%|██████████| 1.75G/1.75G [00:14<00:00, 117MB/s]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split

def sample(x):

  return x.sample(1000)

def load_data(base_dir='SUV-classification2'):
  images = []
  labels = []

  sampled_images = []
  sampled_labels = []


  base = os.path.join(base_dir, 'SUV')

  for category in [ 'Hyundai', 'Kia', ]:
    category_path = os.path.join(base, category) # vehicle-classification/pics/BMW

    for filename in os.listdir(category_path):
      if '.jpg' in filename:
        file_path = os.path.join(category_path, filename)
        images.append(file_path)
        texts = filename.replace('_','$').replace('-','$')
        texts = texts.split('$')
        labels.append(texts[1])


  df_temp = pd.DataFrame({
    'path': images,
    'label': labels
  })

  return df_temp


In [4]:
df_temp = load_data()

data_set = pd.DataFrame()

for g in df_temp['label'].unique():
  if(len(df_temp[df_temp['label'] == g].value_counts())>1000):
    temp_df = df_temp[df_temp['label'] == g].sample(1000)
    data_set = pd.concat([data_set, temp_df])


print(data_set['label'].value_counts())
print(data_set.head())

# 훈련/test 셋 분할

tr_images, test_images, tr_labels, test_labels = train_test_split(data_set['path'], data_set['label'], test_size=0.2, random_state=0)

tr_images.shape, tr_labels.shape, test_images.shape, test_labels.shape  #images path


label
Tucson      1000
Palisade    1000
Santafe     1000
Veracruz    1000
Kona        1000
Sorento     1000
Sportage    1000
Carens      1000
Seltos      1000
Niro        1000
Mohave      1000
Soul        1000
Name: count, dtype: int64
                                                    path   label
16648  SUV-classification2/SUV/Hyundai/SUV_Tucson-860...  Tucson
35337  SUV-classification2/SUV/Hyundai/SUV_Tucson-109...  Tucson
13858  SUV-classification2/SUV/Hyundai/SUV_Tucson-300...  Tucson
3058   SUV-classification2/SUV/Hyundai/SUV_Tucson-689...  Tucson
28764  SUV-classification2/SUV/Hyundai/SUV_Tucson-680...  Tucson


((9600,), (9600,), (2400,), (2400,))

In [6]:
#Xception은 299*299 사이즈로 전달해줘야 함

from tensorflow.keras import applications
pretrained_model = applications.Xception()
pretrained_model.summary()


91884032/91884032 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 149, 149, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 149, 149, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 149, 149, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 147, 147, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 147, 147, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 147, 147, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 147, 147, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 147, 147, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 147, 147, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 147, 147, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 147, 147, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 74, 74, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 74, 74, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 22,910,480 (87.40 MB)

 Trainable params: 22,855,952 (87.19 MB)

 Non-trainable params: 54,528 (213.00 KB)


## 리사이즈 및 파일 저장

In [5]:
from PIL import Image
import os

IMAGE_SIZE = 299
input_dir = 'your_input_directory' # 선별한 디렉토리 구간
output_dir = 'your_output_directory' # 출력할 디렉토리 저장 구간
image_size = (IMAGE_SIZE, IMAGE_SIZE)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 반복문을 통한 리사이즈
for file_name in os.listdir(input_dir):
    if file_name.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path)
        img_resized = img.resize(image_size)
        img_resized.save(os.path.join(output_dir, file_name))



FileNotFoundError: [Errno 2] No such file or directory: 'your_input_directory'

##  Xception 모델 로드

In [ ]:
#Xception은 299*299 사이즈로 전달해줘야 함

from tensorflow.keras import applications
pretrained_model = applications.Xception()
pretrained_model.summary()


In [ ]:
from tensorflow.keras import layers
def create_pretrained_model():
  base_model = applications.Xception(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),include_top=False,)
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(500, activation='relu', kernel_initializer='he_normal')(x)
  x = layers.Dropout(0.3)(x)
  output = layers.Dense(10, activation='softmax')

## 분류를 위한 전이학습

In [ ]:
from tensorflow.keras import layers
def create_pretrained_model():
  base_model = applications.Xception(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),include_top=False,)
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(500, activation='relu', kernel_initializer='he_normal')(x)
  x = layers.Dropout(0.3)(x)
  output = layers.Dense(10, activation='softmax')